In [1]:
import os, sys, time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py
import geopandas as gpd
sys.path.append(os.path.join(os.getcwd(),r'..\..\..'))
# from input_configuration import base_year

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } b
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
base_year = '2023'

In [3]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

In [4]:
pd.options.display.float_format = '{:0,.0f}'.format

# Boardings by Agency

In [5]:
df = pd.read_csv(os.path.join(relative_path,'transit','daily_boardings_by_agency.csv'),index_col=0)
df.loc['Region Total','boardings'] = df['boardings'].sum()
df = df.reset_index()
df.rename(columns={'agency_name': 'Agency', 'boardings': 'Boardings'}, inplace=True)
HTML(df.to_html(index=False))

Agency,Boardings
King County Metro,"288,880"
Sound Transit,"140,400"
Community Transit,"26,595"
Pierce Transit,"23,561"
Kitsap Transit,"10,961"
Washington Ferries,"6,326"
Everett Transit,"5,097"
Region Total,"501,820"


# Boardings by Service Type

In [6]:
df = pd.read_csv(os.path.join(relative_path,'transit','transit_line_results.csv'),index_col=0)
# df['mode'] = df['mode'].map({'b': 'Local Bus','f': 'Ferry','r':'Light Rail','c':'Commuter Rail','p':'Ferry'})
# df = df.groupby('mode').sum()[['boardings']]
# df = df.reset_index()
# df.rename(columns={'mode': 'Service Type', 'boardings': 'Boardings'}, inplace=True)
# HTML(df.to_html(index=False))
transit_type_map = {
    1: 'Local Bus',
    2: 'Express Bus',
    3: 'Bus Rapid Transit',
    4: 'Streetcar',
    5: 'Commuter Rail',
    6: 'Light Rail',
    7: 'Auto Ferry',
    8: 'Passenger Ferry'
}
df['transit_type'] = df['transit_type'].map(transit_type_map)
df = df.groupby('transit_type').sum()[['boardings']]
df = df.reset_index()
df.rename(columns={'boardings':' Boardings','transit_type':'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Boardings
Auto Ferry,"6,326"
Bus Rapid Transit,"64,687"
Commuter Rail,"6,865"
Light Rail,"92,866"
Local Bus,"324,244"
Passenger Ferry,"6,831"


# Trips by Transit Submode
Total numbers of daily trips by transit submode (from Emme trip tables). Trips can include mulitple boardings.

In [7]:
df = pd.read_csv(os.path.join(relative_path,'transit','total_transit_trips.csv'),index_col=0)
df.rename(index={'commuter_rail': 'Commuter Rail',
                'litrat': 'Light Rail',
                'ferry': 'Auto Ferry',
                'passenger_ferry': 'Passenger Ferry',
                'trnst': 'Bus'}, inplace=True)
df.columns = ['Total Trips']
df.loc['Total'] = df.sum(axis=0)
df

,Total Trips
Commuter Rail,"7,669"
Light Rail,"92,037"
Auto Ferry,"8,648"
Passenger Ferry,"3,542"
Bus,"252,169"
Total,"364,066"


# High Capacity Transit (HCT)

In [8]:
# List of Stations
# Load transit stops file

#"The definition we have been using for the RTP is BRT, LRT, Commuter Rail, StreetCar and Ferry."
df = pd.read_csv(r'../../../inputs/scenario/networks/transit_stops.csv')
# Streetcar is coded as light rail
df['hct'] = 0
df.loc[df[['commuter_rail','light_rail','ferry','brt']].sum(axis=1) > 0,'hct'] = 1
df_hct = df[df['hct'] == 1]

In [9]:
# Map of Stations
# Load as a geodataframe
gdf_hct = gpd.GeoDataFrame(
    df_hct, geometry=gpd.points_from_xy(df_hct.x, df_hct.y))

crs = {'init' : 'EPSG:2285'}
gdf_hct.crs = crs

## HCT Station Location

In [10]:
# import contextily as ctx
# ax = gdf_hct.plot(figsize=(20,20), alpha=0.5, edgecolor='k')
# ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, crs=crs)
# ax.set_axis_off()

In [11]:
# Export stops as a shapefile
# gdf_hct.to_file(os.path.join(relative_path,'hct_stops.shp'))

## HCT Network

In [ ]:
df = pd.read_csv(os.path.join(relative_path,'transit','transit_line_results.csv'),index_col=0)
df_hct = df[df['transit_type'].isin([2,3,4,5,6,7,8])]  # see transit_type_map for lookup
# df_segments = pd.read_csv(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_transit_segments.csv')


In [13]:
# df_network = gpd.read_file(r'../../../inputs/scenario/networks/shapefiles/AM/AM_edges.shp')
# hct_segments = df_segments[df_segments['route_id'].isin(df_hct.index)]
# hct_network = df_network[df_network['id'].isin(hct_segments['ij'])]
# hct_network.crs = crs
# ax = hct_network.plot(figsize=(10,20), alpha=0.5, edgecolor='b')
# ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, crs=crs)
# ax.set_axis_off()

## Households Living Near HCT Stations
Within one quarter (1/4) or one half (1/2) miles of HCT station. Results are presented as total households and percent of households within a given geography. 

In [14]:
df_lu = pd.read_csv(r'..\..\..\inputs\scenario\landuse\parcels_urbansim.txt',
                   delim_whitespace=True)

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))

crs = {'init' : 'EPSG:2285'}
gdf_lu.crs = crs

parcel_geog = pd.read_sql_table('parcel_'+base_year+'_geography', 'sqlite:///../../../inputs/db/soundcast_inputs_2023.db')

In [15]:
df_lu = df_lu.merge(parcel_geog,left_on='parcelid', right_on='ParcelID')
# Add a field that defines whether a parcel is inside (1) or outside (0) any RGC
df_lu['RGC_binary'] = 0
df_lu.loc[df_lu['GrowthCenterName'] != 'Not in RGC', 'RGC_binary'] = 1

In [16]:
def calculate_buffer(gdf_lu, gdf_network, distance):
    
    # Buffer the HCT station gdf
    gdf_network['geometry'] = gdf_network.buffer(distance)

    gdf_intersect = gpd.overlay(gdf_network, gdf_lu, how="intersection", keep_geom_type=False)
    df = df_lu[df_lu['parcelid'].isin(gdf_intersect['parcelid'].unique())]

#     df = df.merge(parcel_geog,left_on='PARCELID', right_on='ParcelID')
    
    return df

In [17]:
def aggregate_parcels(df, col_dict, sum_field):
    results_df = pd.DataFrame()
    for col, name in col_dict.items():
        _df = df.groupby(col).sum()[[sum_field]]
        _df['group'] = name
        # results_df = results_df.append(_df)
        results_df = pd.concat([results_df, _df])
    results_df = results_df.reset_index()
    
    return results_df

In [18]:
df_025 = calculate_buffer(gdf_lu, gdf_hct, distance=5280.0/4)

In [19]:
df_050 = calculate_buffer(gdf_lu, gdf_hct, distance=5280.0/2)

In [ ]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {
    # 'racial_geog_vs_reg_total': 'Race',
    #                   'poverty_geog_vs_reg_total': 'Poverty',
    #                     'english_geog_vs_reg_total': 'English',
    #                   'elderly_geog_vs_reg_total': 'Elderly',
    #                   'youth_geog_vs_reg_total': 'Youth',
    #         'disability_geog_vs_reg_total': 'Disability',
    #         'racial_geog_vs_50_percent': 'Race 50%',
    #         'poverty_geog_vs_50_percent': 'Poverty 50%',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'hh_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'hh_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [20]:
pd.options.display.float_format = '{:0,.0f}'.format

### Region
Total households within given distance of HCT stations, regionwide

In [21]:
hct_hh_df['group'].unique()

array(['Race', 'Poverty', 'English', 'Elderly', 'Youth', 'Disability',
       'Race 50%', 'Poverty 50%', 'Regional Geography', 'County',
       'Regional Growth Center', 'RGC Binary', 'Region'], dtype=object)

In [22]:

df = hct_hh_df[hct_hh_df['group'] == 'Region']
df.rename(columns={'HH_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

df['% of Households (1/4 Mile)'] = df['1/4 Mile']/df_lu['HH_P'].sum()
df['% of Households (1/2 Mile)'] = df['1/2 Mile']/df_lu['HH_P'].sum()

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
1,"137,781","395,582",8.6%,24.6%


### Counties
Total households within given distance of HCT stations by county

In [23]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County']
df.rename(columns={'HH_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu.groupby('CountyName').sum()[['HH_P']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['HH_P']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['HH_P']

df = df.merge(_df[['County', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
King,"124,335","315,562",14.1%,35.7%
Kitsap,89,"3,992",0.1%,3.9%
Pierce,"3,061","19,997",0.9%,6.2%
Snohomish,"10,296","56,031",3.5%,19.0%


### Regional Growth Centers

In [24]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary']
df.rename(columns={'HH_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu.groupby('RGC_binary').sum()[['HH_P']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['HH_P']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['HH_P']

df = df.merge(_df[['RGC Binary', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Households (1/4 Mile)', '% of Households (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Outside RGC,"80,822","285,482",5.5%,19.4%
Inside RGC,"56,959","110,100",41.9%,81.1%


In [25]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center']
df.rename(columns={'HH_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

# Calcualte the percent of households in this geography within 1/4 and 1/2 mile

_df = df.merge(df_lu.groupby('GrowthCenterName').sum()[['HH_P']].reset_index(),left_on='Regional Growth Center', 
               right_on='GrowthCenterName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['HH_P']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['HH_P']

df = df.merge(_df[['Regional Growth Center', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Growth Center')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Auburn,311,650,47.8%,100.0%
Ballard-Interbay,124,243,51.0%,100.0%
Bellevue,"2,156","8,239",26.2%,100.0%
Bremerton,46,"1,208",3.6%,95.7%
Burien,"1,297","1,759",73.7%,100.0%
Duwamish,0,372,0.0%,59.9%
Everett,"1,436","3,457",41.5%,100.0%
Federal Way,0,0,NaN,NaN
Kent,334,721,46.3%,100.0%
Kent MIC,0,0,0.0%,0.0%


### Regional Geographies

In [26]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography']
df.rename(columns={'HH_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
# HTML(df.to_html(index=False))

_df = df.merge(df_lu.groupby('rg_proposed').sum()[['HH_P']].reset_index(),left_on='Regional Geography', 
               right_on='rg_proposed')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['HH_P']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['HH_P']

df = df.merge(_df[['Regional Geography', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Geography')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
CitiesTowns,0,501,0.0%,0.4%
Core,"24,203","77,148",6.7%,21.4%
HCT,"10,596","43,406",3.2%,13.0%
Metro,"102,432","271,497",18.9%,50.1%
Rural,41,415,0.0%,0.2%
UU,509,"2,615",0.9%,4.6%


### Equity Geographies

In [27]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df.copy()
df = df[df['index'] == 1]
df.rename(columns={'HH_P': 'Total'}, inplace=True)
df.drop('index', axis=1, inplace=True)
df = df.pivot_table(index='group', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Equity Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


equity_dict = {'racial_geog_vs_reg_total': 'Race',
                      'poverty_geog_vs_reg_total': 'Poverty',
                        'english_geog_vs_reg_total': 'English',
                      'elderly_geog_vs_reg_total': 'Elderly',
                      'youth_geog_vs_reg_total': 'Youth',
            'disability_geog_vs_reg_total': 'Disability',
            'racial_geog_vs_50_percent': 'Race 50%',
            'poverty_geog_vs_50_percent': 'Poverty 50%'}
results_dict = {}
for key, val in equity_dict.items():
    _df = df_lu.groupby(key).sum()[['HH_P']].reset_index()
    results_dict[val] = _df[_df[key] == 1]['HH_P'].values[0]

results_df = pd.DataFrame(results_dict.values(), index=results_dict.keys(), columns=['Total HH'])
results_df = results_df.reset_index()
results_df = results_df.merge(df,left_on='index', right_on='Equity Geography')

results_df['% of Households (1/4 Mile)'] = results_df['1/4 Mile']/results_df['Total HH']
results_df['% of Households (1/2 Mile)'] = results_df['1/2 Mile']/results_df['Total HH']



df = df.merge(results_df[['Equity Geography', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Equity Geography')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Equity Geography,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,"56,236","171,266",7.5%,23.0%
Elderly,"47,304","144,800",6.2%,19.1%
English,"58,395","183,468",9.7%,30.6%
Poverty,"60,981","195,144",9.0%,28.9%
Poverty 50%,"3,916","11,163",10.3%,29.4%
Race,"89,617","230,479",12.5%,32.1%
Race 50%,"40,880","120,236",12.6%,37.0%
Youth,"16,835","83,458",2.1%,10.5%


In [28]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df.copy()
df = df[df['index'] == 0]
df.rename(columns={'HH_P': 'Total'}, inplace=True)
df.drop('index', axis=1, inplace=True)
df = df.pivot_table(index='group', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Equity Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


equity_dict = {'racial_geog_vs_reg_total': 'Race',
                      'poverty_geog_vs_reg_total': 'Poverty',
                        'english_geog_vs_reg_total': 'English',
                      'elderly_geog_vs_reg_total': 'Elderly',
                      'youth_geog_vs_reg_total': 'Youth',
            'disability_geog_vs_reg_total': 'Disability',
            'racial_geog_vs_50_percent': 'Race 50%',
            'poverty_geog_vs_50_percent': 'Poverty 50%'}
results_dict = {}
for key, val in equity_dict.items():
    _df = df_lu.groupby(key).sum()[['HH_P']].reset_index()
    results_dict[val] = _df[_df[key] == 0]['HH_P'].values[0]

results_df = pd.DataFrame(results_dict.values(), index=results_dict.keys(), columns=['Total HH'])
results_df = results_df.reset_index()
results_df = results_df.merge(df,left_on='index', right_on='Equity Geography')

results_df['% of Households (1/4 Mile)'] = results_df['1/4 Mile']/results_df['Total HH']
results_df['% of Households (1/2 Mile)'] = results_df['1/2 Mile']/results_df['Total HH']



df = df.merge(results_df[['Equity Geography', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Equity Geography')
df['Equity Geography'] = ['NOT in '+i for i in df['Equity Geography']]
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Equity Geography,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
NOT in Disability,"81,545","224,316",9.5%,26.1%
NOT in Elderly,"90,477","250,782",10.7%,29.6%
NOT in English,"79,386","212,114",7.9%,21.1%
NOT in Poverty,"76,800","200,438",8.3%,21.5%
NOT in Poverty 50%,"133,865","384,419",8.5%,24.5%
NOT in Race,"48,164","165,103",5.4%,18.6%
NOT in Race 50%,"96,901","275,346",7.6%,21.5%
NOT in Youth,"120,946","312,124",14.9%,38.6%


# Jobs Near HCT Stations
Total jobs and percent of total jobs

### Region

In [29]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'racial_geog_vs_reg_total': 'Race',
                      'poverty_geog_vs_reg_total': 'Poverty',
                        'english_geog_vs_reg_total': 'English',
                      'elderly_geog_vs_reg_total': 'Elderly',
                      'youth_geog_vs_reg_total': 'Youth',
            'disability_geog_vs_reg_total': 'Disability',
            'racial_geog_vs_50_percent': 'Race 50%',
            'poverty_geog_vs_50_percent': 'Poverty 50%',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'EMPTOT_P')
df['distance'] = 0.25
hct_hh_df = hct_hh_df.append(df)
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'EMPTOT_P')
df['distance'] = 0.50
hct_hh_df = hct_hh_df.append(df)

In [30]:
pd.options.display.float_format = '{:0,.0f}'.format

In [31]:

df = hct_hh_df[hct_hh_df['group'] == 'Region']
df.rename(columns={'EMPTOT_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

df['% of Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['EMPTOT_P'].sum()
df['% of Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['EMPTOT_P'].sum()

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
1,"511,166","956,893",24.9%,46.7%


### Counties

In [32]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County']
df.rename(columns={'EMPTOT_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu.groupby('CountyName').sum()[['EMPTOT_P']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['EMPTOT_P']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['EMPTOT_P']

df = df.merge(_df[['County', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
King,"462,417","810,409",32.7%,57.3%
Kitsap,"1,390","14,846",1.7%,17.9%
Pierce,"23,013","66,823",8.0%,23.3%
Snohomish,"24,346","64,815",9.1%,24.3%


### Regional Growth Centers

In [33]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary']
df.rename(columns={'EMPTOT_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu.groupby('RGC_binary').sum()[['EMPTOT_P']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['EMPTOT_P']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['EMPTOT_P']

df = df.merge(_df[['RGC Binary', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
Outside RGC,"111,295","328,802",9.6%,28.5%
Inside RGC,"399,871","628,091",44.6%,70.0%


In [34]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center']
df.rename(columns={'EMPTOT_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['EMPTOT_P'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['EMPTOT_P'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Auburn,"2,311","4,164",0.1%,0.2%
Ballard-Interbay,"7,089","14,773",0.3%,0.7%
Bellevue,"20,456","47,585",1.0%,2.3%
Bremerton,"1,022","3,570",0.0%,0.2%
Burien,"2,400","3,416",0.1%,0.2%
Duwamish,"8,614","32,692",0.4%,1.6%
Everett,"8,706","14,232",0.4%,0.7%
Federal Way,"1,312","2,064",0.1%,0.1%
Kent,"3,375","7,343",0.2%,0.4%
Kent MIC,0,"1,248",0.0%,0.1%


### Regional Geographies

In [35]:
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography']
df.rename(columns={'EMPTOT_P': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['EMPTOT_P'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['EMPTOT_P'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
CitiesTowns,0,519,0.0%,0.0%
Core,"123,606","240,108",6.0%,11.7%
HCT,"14,560","41,017",0.7%,2.0%
Metro,"372,755","674,537",18.2%,32.9%
Rural,115,120,0.0%,0.0%
UU,130,592,0.0%,0.0%


### Equity Geographies


In [36]:
# df = aggregate_parcels(df_lu, col_dict, 'HH_P')
df = hct_hh_df.copy()
df = df[df['index'] == 1]
df.rename(columns={'EMPTOT_P': 'Total'}, inplace=True)
df.drop('index', axis=1, inplace=True)
df = df.pivot_table(index='group', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Equity Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['EMPTOT_P'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['EMPTOT_P'].sum()



output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Equity Geography,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Disability,"312,166","526,861",15.2%,25.7%
Elderly,"237,365","372,282",11.6%,18.2%
English,"215,003","471,686",10.5%,23.0%
Poverty,"323,175","586,199",15.8%,28.6%
Poverty 50%,"12,883","56,180",0.6%,2.7%
RGC Binary,"399,871","628,091",19.5%,30.6%
Race,"378,724","696,059",18.5%,33.9%
Race 50%,"142,598","307,230",7.0%,15.0%
Region,"511,166","956,893",24.9%,46.7%
Youth,"40,718","103,565",2.0%,5.0%


In [37]:
# df = aggregate_parcels(df_lu, col_dict, 'HH_P')
df = hct_hh_df.copy()
df = df[df['index'] == 0]
df.rename(columns={'EMPTOT_P': 'Total'}, inplace=True)
df.drop('index', axis=1, inplace=True)
df = df.pivot_table(index='group', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Equity Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['EMPTOT_P'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['EMPTOT_P'].sum()
df['Equity Geography'] = ['NOT in '+i for i in df['Equity Geography']]
output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Equity Geography,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
NOT in Disability,"199,000","430,032",9.7%,21.0%
NOT in Elderly,"273,801","584,611",13.3%,28.5%
NOT in English,"296,163","485,207",14.4%,23.7%
NOT in Poverty,"187,991","370,694",9.2%,18.1%
NOT in Poverty 50%,"498,283","900,713",24.3%,43.9%
NOT in RGC Binary,"111,295","328,802",5.4%,16.0%
NOT in Race,"132,442","260,834",6.5%,12.7%
NOT in Race 50%,"368,568","649,663",18.0%,31.7%
NOT in Youth,"470,448","853,328",22.9%,41.6%


# Transfers
Transfers between transit modes
- rows are the initial mode and columns are the mode transfered to

In [38]:
df_transfers = pd.read_csv(r'../../../outputs/transit/transit_transfers.csv')

df_line = pd.read_csv(os.path.join(relative_path,'transit','transit_line_results.csv'))

transit_type_map = {
    1: 'Local Bus',
    2: 'Express Bus',
    3: 'Bus Rapid Transit',
    4: 'Streetcar',
    5: 'Commuter Rail',
    6: 'Light Rail',
    7: 'Auto Ferry',
    8: 'Passenger Ferry'
}

transit_type_df = df_line.groupby('line_id').first()[['transit_type']].reset_index()
transit_type_df['transit_type'] = transit_type_df['transit_type'].map(transit_type_map)

df_from = df_transfers.merge(transit_type_df[['transit_type','line_id']], 
                             left_on='from_line_id', right_on='line_id', how='left')
# df = df_from.merge(df_from.merge(df_line[['transit_type']], left_on='to_line_id', right_index=True, how='left',
#                                      suffixes=['_from','_to']))
df = df_from.merge(transit_type_df[['transit_type','line_id']], 
                             left_on='to_line_id', right_on='line_id', how='left', suffixes=['_from','_to'])

In [39]:
_df = df.pivot_table(index='transit_type_from', columns='transit_type_to', values='boardings', aggfunc='sum')
_df = _df.fillna(0)
_df

transit_type_to,Auto Ferry,Bus Rapid Transit,Commuter Rail,Light Rail,Local Bus,Passenger Ferry,Streetcar
transit_type_from,,,,,,,
Auto Ferry,230,835,14,154,"4,462",187,0
Bus Rapid Transit,"1,238","3,536",769,"4,892","13,354",199,2
Commuter Rail,7,391,25,339,"9,905",16,18
Light Rail,107,"4,782",440,61,"26,199",172,28
Local Bus,"3,804","12,179","9,070","21,962","102,775","1,689",119
Passenger Ferry,244,126,24,76,"1,857",722,0
Streetcar,0,0,4,286,429,0,0


#### Transit Mode for Transfers to HCT Routes
e.g., rode local bus to HCT route

In [40]:
# Total number of transfers to HCT from transit modes
_df = df.groupby(['transit_type_from','transit_type_to']).sum()[['boardings']].reset_index()

_df.groupby('transit_type_from').sum()[['boardings']]

,boardings
transit_type_from,
Auto Ferry,"5,882"
Bus Rapid Transit,"23,991"
Commuter Rail,"10,702"
Light Rail,"31,790"
Local Bus,"151,598"
Passenger Ferry,"3,049"
Streetcar,719


#### Transit Mode for Transfers from HCT Routes
e.g., took local bus after using HCT route

In [41]:
# Total number of transfers to HCT from transit modes
_df.groupby('transit_type_to').sum()[['boardings']]

,boardings
transit_type_to,
Auto Ferry,"5,631"
Bus Rapid Transit,"21,850"
Commuter Rail,"10,346"
Light Rail,"27,770"
Local Bus,"158,982"
Passenger Ferry,"2,985"
Streetcar,168


# Transit Access Mode
### Percent of Transit Trips Accessed by Walking

#### Region

In [42]:
pd.options.display.float_format = '{:0,.1%}'.format
df = pd.read_csv(r'../../../outputs/agg/dash/trip_mode_by_tour_mode.csv')
_df = df.copy()
pnr_transit_trips = _df[(_df['tmodetp'] == 'Park') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
walk_transit_trips = _df[(_df['tmodetp'] == 'Transit') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
reg_access = (walk_transit_trips/(walk_transit_trips+pnr_transit_trips))
pd.DataFrame([reg_access], columns=['% Transit trips Accessed by Walking'], index=['Region'])


,% Transit trips Accessed by Walking
Region,83.3%


#### Equity Geographies

In [43]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = df.copy()
results_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
for col in ['hh_disability','hh_elderly','hh_english','hh_poverty','hh_racial','hh_youth']:
    for suffix in ['_reg', '_50']:
        pnr_transit_trips = _df[(_df[col+suffix] == 1) & 
                                (_df['tmodetp'] == 'Park') &
                                (_df['mode'] == 'Transit')]['trexpfac'].sum()
        walk_transit_trips = _df[(_df[col+suffix] == 1) & 
                                  (_df['tmodetp'] == 'Transit') & 
                                  (_df['mode'] == 'Transit')]['trexpfac'].sum()
        if (pnr_transit_trips > 0) & (walk_transit_trips > 0):
            if suffix == '_reg' or col in list_50:
                results_df.loc[col+suffix,'% Transit Trips Accessed by Walking'] = walk_transit_trips/(walk_transit_trips+pnr_transit_trips)
# results_df.rename(columns={'_reg':'> Regional Average', '_50': '> 50%'}, inplace=True)
results_df

,% Transit Trips Accessed by Walking
hh_disability_reg,83.5%
hh_elderly_reg,82.6%
hh_english_reg,84.8%
hh_poverty_reg,87.9%
hh_poverty_50,92.3%
hh_racial_reg,85.9%
hh_racial_50,88.3%
hh_youth_reg,74.2%


In [44]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = df.copy()
results_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
for col in ['hh_disability','hh_elderly','hh_english','hh_poverty','hh_racial','hh_youth']:
    for suffix in ['_reg', '_50']:
        pnr_transit_trips = _df[(_df[col+suffix] == 0) & 
                                (_df['tmodetp'] == 'Park') &
                                (_df['mode'] == 'Transit')]['trexpfac'].sum()
        walk_transit_trips = _df[(_df[col+suffix] == 0) & 
                                  (_df['tmodetp'] == 'Transit') & 
                                  (_df['mode'] == 'Transit')]['trexpfac'].sum()
        if (pnr_transit_trips > 0) & (walk_transit_trips > 0):
            if suffix == '_reg' or col in list_50:
                results_df.loc[col+suffix,'% Transit Trips Accessed by Walking'] = walk_transit_trips/(walk_transit_trips+pnr_transit_trips)
# results_df.rename(columns={'_reg':'> Regional Average', '_50': '> 50%'}, inplace=True)
results_df.index = ['NOT in '+i for i in results_df.index]
results_df

,% Transit Trips Accessed by Walking
NOT in hh_disability_reg,83.2%
NOT in hh_elderly_reg,83.9%
NOT in hh_english_reg,82.1%
NOT in hh_poverty_reg,79.1%
NOT in hh_poverty_50,83.0%
NOT in hh_racial_reg,80.4%
NOT in hh_racial_50,81.3%
NOT in hh_youth_reg,88.8%
